In [ ]:
import 

In [4]:
import json
import re

# Load the data
with open(r"C:\Users\DELL\Downloads\dish_names.json") as file:
    data = json.load(file)

# Function to clean a list of ingredients
def clean_ingredient_list(ingredient_list):
    cleaned = []
    for ingredient in ingredient_list:
        cleaned_ing = re.sub(r'\([^)]*\)', '', ingredient)  # remove text inside brackets
        cleaned_ing = re.sub(r'\b\d+[%]?\b', '', cleaned_ing)  # remove percentages and digits
        cleaned_ing = re.sub(
            r'\b\d+\s?(oz|lb|g|ml|cup|cups|tablespoons|tbsp|teaspoons|tsp|pkg|can|cans|pound|pounds)\b',
            '', cleaned_ing, flags=re.IGNORECASE)
        cleaned_ing = re.sub(r'[^a-zA-Z\s]', '', cleaned_ing)  # remove special characters
        cleaned_ing = cleaned_ing.strip().lower()
        if cleaned_ing:
            cleaned.append(cleaned_ing)
    return cleaned

# Apply cleaning to all entries
for entry in data:
    entry["cleaned_ingredients"] = clean_ingredient_list(entry.get("ingredients", []))

# Save cleaned data if needed
with open("cleaned_dishes.json", "w") as outfile:
    json.dump(data, outfile, indent=2)





In [5]:
# Optional: print first 3 cleaned samples
for entry in data[:3]:
    print("Dish:", entry["dish_name"])
    print("Cleaned Ingredients:", entry["cleaned_ingredients"])

Dish: Greek Salad
Cleaned Ingredients: ['romaine lettuce', 'black olives', 'grape tomatoes', 'garlic', 'pepper', 'purple onion', 'seasoning', 'garbanzo beans', 'feta cheese crumbles']
Dish: Southern_Us Special
Cleaned Ingredients: ['plain flour', 'ground pepper', 'salt', 'tomatoes', 'ground black pepper', 'thyme', 'eggs', 'green tomatoes', 'yellow corn meal', 'milk', 'vegetable oil']
Dish: Filipino Special
Cleaned Ingredients: ['eggs', 'pepper', 'salt', 'mayonaise', 'cooking oil', 'green chilies', 'grilled chicken breasts', 'garlic powder', 'yellow onion', 'soy sauce', 'butter', 'chicken livers']


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

# Prepare data for model
dishes = []
ingredients_corpus = []

for entry in data:
    if entry["cleaned_ingredients"]:
        ingredients_text = " ".join(entry["cleaned_ingredients"])
        ingredients_corpus.append(ingredients_text)
        dishes.append(entry["dish_name"])

# Convert ingredients to feature vectors
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(ingredients_corpus)
y = dishes

# Split data (optional - for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train KNN model
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# Predict function
def predict_dish(input_ingredients):
    cleaned_input = clean_ingredient_list(input_ingredients)
    input_text = " ".join(cleaned_input)
    input_vector = vectorizer.transform([input_text])
    prediction = knn.predict(input_vector)
    return prediction[0]



In [8]:
# Example usage
user_input = ["garlic", "lemon juice", "romaine lettuce", "black olives", "olive oil", "feta cheese"]
predicted_dish = predict_dish(user_input)
print("Predicted Dish:", predicted_dish)


Predicted Dish: Italian Special


In [9]:
# Save cleaned data to JSON file in the same directory
with open("cleaned_dishes.json", "w") as outfile:
    json.dump(data, outfile, indent=2)
